# Sweep Experiment with Metrics

TJ Kim <br/>
2/27/21

Updated <br/>
2/27/21

#### Objective: 
Run experiments given new transferability metrics where we sweep across the following parameters: <br/>

Inputs:<br/>
- Number of Layers
- Number of iterations (FGSM)
- Confidence Parameter (C&W)

Outputs:<br/>
- Old Transferability Metrics (Label Based)
- New Transferability Metrics (Empirical)

Also, print outputs to excel files in a organized fashion (tables).

In [1]:
cd '/home/ubuntu/FedAtk/' 

/home/ubuntu/FedAtk


### Load Relevant Libraries and Modules

Load the relevant libraries for the federated learning code.

In [2]:
# Transferer
from transfer_attacks.Transferer import *
from configs.overwrite_config import *


import torch
import numpy as np
import os
import pandas as pd

%load_ext autoreload
%autoreload 2

### Organize Experiment Parameters

Sort the following:<br/>
- Names of the different layers
- Save folder for csv files

In [3]:
# Make directory in results for this experiment
exp_name = '21_3_2 Experiment 1'
exp_path = "results/" + exp_name
if not os.path.isdir(exp_path):
    os.mkdir(exp_path)
    
# FL Architecture
client_idx = 0
victim_idxs = [0,1,2,3,4,5,6,7]

# Saved Neural Networks to Test on 
exp_names = ["exp4_neck2_0_head3",
             "exp4_neck2_1_head2",
             "exp4_neck2_2_head1"]

# Parameters to record for excel printing
num_clients = 8
metrics = ['orig_acc','orig_sim','adv_sim','adv_hit','g_align']
ifgsm_iterations = [1,10,20,30]
cw_confidences = [0, 5, 10, 20]

# Save 1 - neck2_head3 network per client metric storage
stored_per_client_fgsm = {}
stored_per_client_cw = {}
stored_per_client_fgsm['num_clients'] = np.arange(num_clients)
stored_per_client_cw['num_clients'] = np.arange(num_clients)
for i in metrics:
    stored_per_client_fgsm[i] = np.zeros(num_clients)
    stored_per_client_cw[i] = np.zeros(num_clients)

# Save 2 - Across all networks
stored_per_layer_fgsm = {}
stored_per_layer_cw = {}
stored_per_layer_fgsm['exp_name'] = exp_names
stored_per_layer_cw['exp_name'] = exp_names
for i in metrics:
    stored_per_layer_fgsm[i] = np.zeros(len(exp_names))
    stored_per_layer_cw[i] = np.zeros(len(exp_names))
    
# Save 3 - neck2_head3 ifsgm iteration sweep
stored_fgsm_iteration = {}
stored_fgsm_iteration['iterations'] = ifgsm_iterations
for i in metrics:
    stored_fgsm_iteration[i] = np.zeros(len(ifgsm_iterations))

# Save 4 - neck2_head3 CW confidence sweep
stored_cw_confidence = {}
stored_cw_confidence['confidence'] = cw_confidences
for i in metrics:
    stored_cw_confidence[i] = np.zeros(len(cw_confidences))

### Run the experiments

Load different experiments and fill in the tables.

First run first simulation regarding checking different servers.

In [4]:
# Experiment 1 - Per Client transfer metrics

# Generate NN and Victims
transferer = Transferer(filename = exp_names[0])
transferer.generate_advNN(client_idx = client_idx)
transferer.generate_victims(client_idxs = victim_idxs)
print('generated model')

# FGSM Attack
print('starting experiment 1')
transferer.generate_xadv(atk_type = "ifsgm")
transferer.send_to_victims(victim_idxs)
transferer.check_empirical_metrics(orig_flag = True)

# Log Values per server
for i in range(num_clients):
    stored_per_client_fgsm['orig_acc'][i] = transferer.orig_acc_transfers[i]
    stored_per_client_fgsm['orig_sim'][i] = transferer.orig_similarities[i]
    stored_per_client_fgsm['adv_sim'][i] = transferer.adv_similarities[i]
    stored_per_client_fgsm['adv_hit'][i] = transferer.adv_target_hit[i]
    stored_per_client_fgsm['g_align'][i] = transferer.metric_alignment[i]

print('finished IFGSM Attack')

# CW Attack
transferer.generate_xadv(atk_type = "cw")
transferer.send_to_victims(victim_idxs)
transferer.check_empirical_metrics(orig_flag = True)

# Log Values per server
for i in range(num_clients):
    stored_per_client_cw['orig_acc'][i] = transferer.orig_acc_transfers[i]
    stored_per_client_cw['orig_sim'][i] = transferer.orig_similarities[i]
    stored_per_client_cw['adv_sim'][i] = transferer.adv_similarities[i]
    stored_per_client_cw['adv_hit'][i] = transferer.adv_target_hit[i]
    stored_per_client_cw['g_align'][i] = transferer.metric_alignment[i]

print('finished CW Attack')


# Experiment 3 - FGSM Iteration sweep
print("starting experiment 3")
for it in range(len(ifgsm_iterations)):
    transferer.ifsgm_params.set_params(iteration = ifgsm_iterations[it])
    transferer.generate_xadv(atk_type = "ifsgm")
    transferer.send_to_victims(victim_idxs)
    transferer.check_empirical_metrics(orig_flag = True)
    
    stored_fgsm_iteration['orig_acc'][it] = sum(transferer.orig_acc_transfers.values()) / len(transferer.orig_acc_transfers) 
    stored_fgsm_iteration['orig_sim'][it] = sum(transferer.orig_similarities.values())/ len(transferer.orig_similarities)
    stored_fgsm_iteration['adv_sim'][it] = sum(transferer.adv_similarities.values())/ len(transferer.adv_similarities)
    stored_fgsm_iteration['adv_hit'][it] = sum(transferer.adv_target_hit.values())/len(transferer.adv_target_hit)
    stored_fgsm_iteration['g_align'][it] = sum(transferer.metric_alignment.values())/len(transferer.metric_alignment)
    
    print('finished fgsm iter sweep', it)


# Experiment 4 - CW Confidence Sweep
print("starting experiment 4")
for c in range(len(cw_confidences)):
    transferer.cw_params.set_params(confidence = cw_confidences[c])
    transferer.generate_xadv(atk_type = "cw")
    transferer.send_to_victims(victim_idxs)
    transferer.check_empirical_metrics(orig_flag = True)
    
    stored_cw_confidence['orig_acc'][c] = sum(transferer.orig_acc_transfers.values()) / len(transferer.orig_acc_transfers) 
    stored_cw_confidence['orig_sim'][c] = sum(transferer.orig_similarities.values())/ len(transferer.orig_similarities)
    stored_cw_confidence['adv_sim'][c] = sum(transferer.adv_similarities.values())/ len(transferer.adv_similarities)
    stored_cw_confidence['adv_hit'][c] = sum(transferer.adv_target_hit.values())/len(transferer.adv_target_hit)
    stored_cw_confidence['g_align'][c] = sum(transferer.metric_alignment.values())/len(transferer.metric_alignment)
    
    print('finished cw confidence sweep', c)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Loading  all_data_12_niid_0_keep_0_train_9.json
Loading  all_data_20_niid_0_keep_0_train_9.json
Loading  all_data_11_niid_0_keep_0_train_9.json
Loading  all_data_18_niid_0_keep_0_train_9.json


/home/ubuntu/anaconda3/lib/python3.8/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


load_FLNN exppath: federated_training/Results/federated_system/exp4_neck2_0_head3/exp4_neck2_0_head3_
load_FLNN exppath: federated_training/Results/federated_system/exp4_neck2_0_head3/exp4_neck2_0_head3_
load_FLNN exppath: federated_training/Results/federated_system/exp4_neck2_0_head3/exp4_neck2_0_head3_
load_FLNN exppath: federated_training/Results/federated_system/exp4_neck2_0_head3/exp4_neck2_0_head3_
load_FLNN exppath: federated_training/Results/federated_system/exp4_neck2_0_head3/exp4_neck2_0_head3_
load_FLNN exppath: federated_training/Results/federated_system/exp4_neck2_0_head3/exp4_neck2_0_head3_
load_FLNN exppath: federated_training/Results/federated_system/exp4_neck2_0_head3/exp4_neck2_0_head3_
load_FLNN exppath: federated_training/Results/federated_system/exp4_neck2_0_head3/exp4_neck2_0_head3_
load_FLNN exppath: federated_training/Results/federated_system/exp4_neck2_0_head3/exp4_neck2_0_head3_
generated model
starting experiment 1
finished IFGSM Attack
finished CW Attack
sta

2nd Simulation regarding same metrics across different layer shared between servers.

In [5]:
# Experiment 2 - Per Layer Transfer Metric
print("Running Exp 2")

for l in range(len(exp_names)):
    # Generate NN and Victims
    transferer = Transferer(filename = exp_names[l])
    transferer.generate_advNN(client_idx = client_idx)
    transferer.generate_victims(client_idxs = victim_idxs)
    print('generated model', l)

    # FGSM Attack
    transferer.generate_xadv(atk_type = "ifsgm")
    transferer.send_to_victims(victim_idxs)
    transferer.check_empirical_metrics(orig_flag = True)

    # Log Values per layer
    stored_per_layer_fgsm['orig_acc'][l] = sum(transferer.orig_acc_transfers.values()) / len(transferer.orig_acc_transfers) 
    stored_per_layer_fgsm['orig_sim'][l] = sum(transferer.orig_similarities.values())/ len(transferer.orig_similarities)
    stored_per_layer_fgsm['adv_sim'][l] = sum(transferer.adv_similarities.values())/ len(transferer.adv_similarities)
    stored_per_layer_fgsm['adv_hit'][l] = sum(transferer.adv_target_hit.values())/len(transferer.adv_target_hit)
    stored_per_layer_fgsm['g_align'][l] = sum(transferer.metric_alignment.values())/len(transferer.metric_alignment)

    print('finished IFGSM Attack', l)

    # CW Attack
    transferer.generate_xadv(atk_type = "cw")
    transferer.send_to_victims(victim_idxs)
    transferer.check_empirical_metrics(orig_flag = True)

    # Log Values per server
    stored_per_layer_cw['orig_acc'][l] = sum(transferer.orig_acc_transfers.values()) / len(transferer.orig_acc_transfers) 
    stored_per_layer_cw['orig_sim'][l] = sum(transferer.orig_similarities.values())/ len(transferer.orig_similarities)
    stored_per_layer_cw['adv_sim'][l] = sum(transferer.adv_similarities.values())/ len(transferer.adv_similarities)
    stored_per_layer_cw['adv_hit'][l] = sum(transferer.adv_target_hit.values())/len(transferer.adv_target_hit)
    stored_per_layer_cw['g_align'][l] = sum(transferer.metric_alignment.values())/len(transferer.metric_alignment)

    print('finished CW Attack', l)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Loading  all_data_12_niid_0_keep_0_train_9.json
Loading  all_data_20_niid_0_keep_0_train_9.json
Loading  all_data_11_niid_0_keep_0_train_9.json
Loading  all_data_18_niid_0_keep_0_train_9.json
generated model 0
finished IFGSM Attack 0
finished CW Attack 0
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Loading  all_data_12_niid_0_keep_0_train_9.json
Loading  all_data_20_niid_0_keep_0_train_9.json
Loading  all_data_11_niid_0_keep_0_train_9.json
Loading  all_data_18_niid_0_keep_0_train_9.json
generated model 1
finished IFGSM Attack 1
finished CW Attack 1
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Loading  all_data_12_niid_0_keep_0_train_9.json
Loading  all_data_20_niid_0_keep_0_train_9.json
Loading  all_data_11_niid_0_keep_0_train_9.json
Loading  all_data_18_niid_0_keep_0_train_9.json
generated model 2
finished IFGSM A

### Write results to Excel file

One excel file for each dictionary (6 in total).

In [6]:
# Save 1 - neck2_head3 network per client metric storage
df1_1 = pd.DataFrame(data=stored_per_client_fgsm).T
df1_1.to_excel(exp_path + '/'+'s1_1_per_client_fgsm.xlsx',header=False)

df1_2 =  pd.DataFrame(data=stored_per_client_cw).T
df1_2.to_excel(exp_path + '/'+'s1_2_per_client_cw.xlsx',header=False)

# Save 3 - neck2_head3 ifsgm iteration sweep
df3 = pd.DataFrame(data=stored_fgsm_iteration).T
df3.to_excel(exp_path + '/' + 's3_fgsm_iterations.xlsx',header=False)

# Save 4 - neck2_head3 CW confidence sweep
df4 = pd.DataFrame(data=stored_cw_confidence).T
df4.to_excel(exp_path + '/' + 's4_cw_confidence.xlsx', header=False)

In [7]:
# Save 2 - Across all networks
df2_1 = pd.DataFrame(data=stored_per_layer_fgsm).T
df2_1.to_excel(exp_path + '/'+'s2_1_per_layer_fgsm.xlsx',header=False)

df2_2 = pd.DataFrame(data=stored_per_layer_cw).T
df2_2.to_excel(exp_path + '/'+'s2_2_per_layer_cw.xlsx',header=False)